In [1]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import numpy as np
import lightgbm as lgb
# from calc_uauc import uAUC
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from scipy import sparse
import os
import warnings
from imblearn.under_sampling import RandomUnderSampler
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 100)
tqdm.pandas(desc='pandas bar')

def get_count(df, column, feature):
    df['idx'] = range(len(df))
    temp = df.groupby(column)['userid'].agg([(feature, 'count')]).reset_index()
    df = df.merge(temp)
    df = df.sort_values('idx').drop('idx', axis=1).reset_index(drop=True)
    return df


def get_sum(df, column, feature):
    df['idx'] = range(len(df))
    temp = df.groupby(column)['userid'].agg([(feature, 'sum')]).reset_index()
    df = df.merge(temp)
    df = df.sort_values('idx').drop('idx', axis=1).reset_index(drop=True)
    return df


def reduce_mem(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df

In [2]:
test = pd.read_csv('./phm_test.csv', header=None)
test.columns = ['time','gun_no','C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force','W_Start_friction','W_us2','W_Welding_point_count','W_position_count','area','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up']
test = reduce_mem(test)

503.54 Mb, 165.94 Mb (67.05 %)


In [3]:
train = pd.DataFrame(columns = ['time','gun_no','W_Error','C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force','W_Start_friction','W_us2','W_Welding_point_count','W_position_count','area','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up', 'lifetime'])
#train.columns = ['time','gun_no','W_Error','C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force','W_Start_friction','W_us2','W_Welding_point_count','W_position_count','area','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
#'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up']
for i in os.listdir("./task3"):
    if i == "gun4.csv":
        continue
    temp = pd.read_csv('./task3/'+i)
    train = pd.concat([train, temp], ignore_index=True)
train = reduce_mem(train)
train.head()

826.34 Mb, 645.58 Mb (21.87 %)


,time,gun_no,W_Error,C_Cylinder_force,C_Differential_pressure,W_Friction,W_Maximum_aperture,W_Maximum_electrode_force,W_Start_friction,W_us2,W_Welding_point_count,W_position_count,area,in_Counterbalance_pressure,in_Electrode_force,in_Electrode_position,in_Sheet_thickness,in_Velocity,in_force_build_up,out_Cap_offset,out_Electrode_force,out_Electrode_position,out_force_build_up,lifetime
0,1620518400,0,0,-64,1.089844,-57,152.75,5013,296,0,299981,12169036,MF,14.992188,3500,150.0,2.0,1,0,2.400391,0,150.25,0,432000
1,1620518401,0,0,-64,1.089844,-57,152.75,5013,296,0,299981,12169036,MF,14.992188,3500,150.0,2.0,1,0,2.400391,0,150.25,0,432000
2,1620518402,0,0,-64,1.089844,-57,152.75,5013,296,0,299981,12169036,MF,14.992188,3500,150.0,2.0,1,0,2.400391,0,150.25,0,432000
3,1620518403,0,0,-64,1.089844,-57,152.75,5013,296,0,299981,12169036,MF,14.992188,3500,150.0,2.0,1,0,2.400391,0,150.25,0,432000
4,1620518404,0,0,-64,1.089844,-57,152.75,5013,296,0,299981,12169036,MF,14.992188,3500,150.0,2.0,1,0,2.400391,0,150.25,0,432000


In [4]:
train_drop_columns = ['time', 'gun_no', 'W_Error', 'area']
train.drop(train_drop_columns, axis=1, inplace=True)
test_drop_columns = ['time', 'gun_no', 'area']
test.drop(test_drop_columns, axis=1, inplace=True)
print(len(train))

4512913


In [5]:
train = train.drop_duplicates(['C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force','W_Start_friction','W_us2','W_Welding_point_count','W_position_count','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up'])
print(len(train))

2307780


In [25]:
from sklearn.model_selection import KFold
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
folds = KFold(n_splits=5, shuffle=True, random_state=2019)
oof_lgb = np.zeros(len(train))
feature_importance_df = pd.DataFrame()
col = [tmp_col for tmp_col in train.columns if tmp_col not in ['lifetime']]
#print(train[col].head())
print(train[col].head())
X_train = train[col].values
# print(train[col].info())
y_train = train['lifetime'].values
print(y_train.mean())

340762.93449072266


In [26]:
lgb_params = {
                        'boosting_type': 'gbdt',
                        'objective': 'regression',
                        'num_leaves': 40,
                        'subsample': 0.9,
                        'learning_rate': 0.25,
                        'seed': 2017,
                        'nthread': -1,
                        'metrics':'mse'
             }

In [27]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_ + 1))
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    num_round = 2000
    clf = lgb.train(lgb_params, trn_data, num_round, valid_sets=[trn_data, val_data], verbose_eval=20,
                    early_stopping_rounds=100)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = col
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
# oof_lgb2 += clf.predict(df_remove[col].values, num_iteration=clf.best_iteration)
# oof_lgb_final = np.argmax(oof_lgb, axis=1)
#     predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits
print('feature importance:')
print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(15))

fold n°1
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.075324 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2051
[LightGBM] [Info] Number of data points in the train set: 1846224, number of used features: 19
[LightGBM] [Info] Start training from score 340774.165834
Training until validation scores don't improve for 100 rounds
[20]	training's l2: 1.39681e+07	valid_1's l2: 1.39636e+07
[40]	training's l2: 7.03036e+06	valid_1's l2: 7.22682e+06
[60]	training's l2: 4.46771e+06	valid_1's l2: 4.93391e+06
[80]	training's l2: 3.47401e+06	valid_1's l2: 3.97331e+06
[100]	training's l2: 2.83848e+06	valid_1's l2: 3.38554e+06
[120]	training's l2: 2.42371e+06	valid_1's l2: 3.06655e+06
[140]	training's l2: 2.11282e+06	valid_1's l2: 2.74871e+06
[160]	training's l2: 1.83383e+06	valid_1's l2: 2.45961e+06
[180]	training's l2: 1.66141e+06	valid_1's l2

[580]	training's l2: 885217	valid_1's l2: 1.17227e+06
[600]	training's l2: 871851	valid_1's l2: 1.16569e+06
[620]	training's l2: 853387	valid_1's l2: 1.15433e+06
[640]	training's l2: 838538	valid_1's l2: 1.14838e+06
[660]	training's l2: 825675	valid_1's l2: 1.14722e+06
[680]	training's l2: 814773	valid_1's l2: 1.1419e+06
[700]	training's l2: 804689	valid_1's l2: 1.13791e+06
[720]	training's l2: 796040	valid_1's l2: 1.13731e+06
[740]	training's l2: 788006	valid_1's l2: 1.13554e+06
[760]	training's l2: 776307	valid_1's l2: 1.13003e+06
[780]	training's l2: 762424	valid_1's l2: 1.13159e+06
[800]	training's l2: 754652	valid_1's l2: 1.12773e+06
[820]	training's l2: 746749	valid_1's l2: 1.12703e+06
[840]	training's l2: 735712	valid_1's l2: 1.12454e+06
[860]	training's l2: 727327	valid_1's l2: 1.12207e+06
[880]	training's l2: 722069	valid_1's l2: 1.12165e+06
[900]	training's l2: 714640	valid_1's l2: 1.11984e+06
[920]	training's l2: 703065	valid_1's l2: 1.11747e+06
[940]	training's l2: 694818	v

[1800]	training's l2: 426301	valid_1's l2: 783394
[1820]	training's l2: 425216	valid_1's l2: 783122
[1840]	training's l2: 422445	valid_1's l2: 783044
[1860]	training's l2: 418930	valid_1's l2: 781681
[1880]	training's l2: 417869	valid_1's l2: 781085
[1900]	training's l2: 415513	valid_1's l2: 781476
[1920]	training's l2: 414073	valid_1's l2: 780722
[1940]	training's l2: 412658	valid_1's l2: 780394
[1960]	training's l2: 410489	valid_1's l2: 781241
[1980]	training's l2: 407942	valid_1's l2: 782235
[2000]	training's l2: 407121	valid_1's l2: 782282
Did not meet early stopping. Best iteration is:
[2000]	training's l2: 407121	valid_1's l2: 782282
fold n°4
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.076191 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2052
[LightGBM] [Info] Number of data points in the train set: 1846224, number of use

[1200]	training's l2: 584428	valid_1's l2: 1.48932e+06
[1220]	training's l2: 581382	valid_1's l2: 1.48994e+06
[1240]	training's l2: 575095	valid_1's l2: 1.48725e+06
[1260]	training's l2: 571973	valid_1's l2: 1.4886e+06
[1280]	training's l2: 567412	valid_1's l2: 1.48829e+06
[1300]	training's l2: 564242	valid_1's l2: 1.4853e+06
[1320]	training's l2: 560600	valid_1's l2: 1.4851e+06
[1340]	training's l2: 557118	valid_1's l2: 1.48535e+06
[1360]	training's l2: 554419	valid_1's l2: 1.48523e+06
[1380]	training's l2: 549755	valid_1's l2: 1.48404e+06
[1400]	training's l2: 545331	valid_1's l2: 1.48347e+06
[1420]	training's l2: 542752	valid_1's l2: 1.48132e+06
[1440]	training's l2: 540343	valid_1's l2: 1.48071e+06
[1460]	training's l2: 537417	valid_1's l2: 1.48058e+06
[1480]	training's l2: 535164	valid_1's l2: 1.47903e+06
[1500]	training's l2: 532456	valid_1's l2: 1.47509e+06
[1520]	training's l2: 529208	valid_1's l2: 1.47277e+06
[1540]	training's l2: 526532	valid_1's l2: 1.4728e+06
[1560]	trainin

In [37]:
predict = clf.predict(test, num_iteration=clf.best_iteration)
print(predict.mean())
print(type(predict))

284806.9621465048
<class 'numpy.ndarray'>


In [44]:
result = pd.DataFrame(predict, dtype='int64')
print(result.head())
result[result[0] > 432000] = 432000.
result[result[0] < 0] = 0
print(len(result))
result.to_csv("./result/predict_rul.csv", header=0, index=0)

         0
0  4320000
1   431937
2   431942
3  4320000
4   431938
3000000


In [18]:
predictions = np.zeros(len(test))
for i, (train_index, val_index) in enumerate(skf.split(X_train,y_train)):
    print("fold {}".format(i))
    X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    lgb_train = lgb.Dataset(X_tr,y_tr)
    lgb_val = lgb.Dataset(X_val,y_val)
    num_round = 2000
    clf = lgb.train(lgb_params, lgb_train, num_round, valid_sets = [lgb_train, lgb_val],verbose_eval=50, 
                        early_stopping_rounds = 50)
    print('best iteration = ',clf.best_iteration)
    print("*"*100)
    predictions += clf.predict(test, num_iteration=clf.best_iteration) / skf.n_splits

fold 0


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: C_Cylinder_force, W_Friction, W_Maximum_electrode_force, W_Start_friction, W_us2, W_Welding_point_count, W_position_count, in_Electrode_force, in_Velocity, in_force_build_up, out_Electrode_force, out_force_build_up